# Tutorial: Always Encrypted


<span style="color: var(--vscode-foreground);">This notebook uses the sample AdventureWorks sample database but can be used with any user database.</span>

# Step 1: Enable Always Encrypted with secure enclaves in SQL Server

## In this step, you'll enable the functionality of Always Encrypted using enclaves in your SQL Server instance.

## 1.  Using SSMS, connect to your SQL Server instance as sysadmin **without** Always Encrypted enabled for the database connection.

## 

- Start SSMS.
- In the **Connect to Server** dialog, specify your server name, select an authentication method and specify your credentials.
- Select **Options \>** and select the **Always Encrypted tab**.
- Make sure the **Enable Always Encrypted (column encryption)** checkbox is **not** selected.
- Select **Connect**.

## 2.  Open a new query window, and execute the below statement to set the secure enclave type to virtualization based security (VBS).

In [ ]:
EXEC sys.sp_configure 'column encryption enclave type', 1;
RECONFIGURE;

## 3.  Restart your SQL Server instance for the previous change to take effect. You can restart the instance in SSMS by right-clicking on it in Object Explorer and selecting Restart. Once the instance restarts, reconnect to it.

## 4.  Confirm the secure enclave is now loaded by running the following query:

In [ ]:
SELECT [name], [value], [value_in_use] FROM sys.configurations
WHERE [name] = 'column encryption enclave type';

# Step 2: Create a table with no encrytion

In [ ]:
USE AdventureWorks
GO

IF  EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[HumanResources].[Employees]') AND type in (N'U'))
DROP TABLE [HumanResources].[Employees]
GO

CREATE TABLE [HumanResources].[Employees]
(
    [EmployeeID] [int] IDENTITY(1,1) NOT NULL
    , [SSN] [char](11) NOT NULL
    , [FirstName] [nvarchar](50) NOT NULL
    , [LastName] [nvarchar](50) NOT NULL
    , [Salary] [money] NOT NULL
) ON [PRIMARY];



# Step 3: Add a few records

In [ ]:
INSERT INTO [HumanResources].[Employees]
(
    [SSN]
    , [FirstName]
    , [LastName]
    , [Salary]
)
VALUES
(
    '795-73-9838'
    , N'Catherine'
    , N'Abel'
    , $31692
);

INSERT INTO [HumanResources].[Employees]
(
    [SSN]
    , [FirstName]
    , [LastName]
    , [Salary]
)
VALUES
(
    '990-00-6818'
    , N'Kim'
    , N'Abercrombie'
    , $55415
);

# Step 4: Provision enclave-enabled keys

In this step, you'll create a column master key and a column encryption key that allow enclave computations.

1. Using the SSMS instance from the previous step, in **Object Explorer**, expand your database and navigate to **Security** \> **Always Encrypted Keys**.
    
2. Provision a new enclave-enabled column master key:
    

- Right-click **Always Encrypted Keys** and select **New Column Master Key...**.
    
- Select your column master key name: **CMK1**.
    
- Make sure you select either **Windows Certificate Store (Current User or Local Machine)**
    
- Select **Allow enclave computations**
    
- Select your certificate if it already exists, or select the **Generate Certificate** button to create a new one.
    
- Select **OK**.
    

3.  Create a new enclave-enabled column encryption key:

- Right-click **Always Encrypted Keys** and select **New Column Encryption Key**.
- Enter a name for the new column encryption key: **CEK1**.
- In the **Column master key** dropdown, select the column master key you created in the previous steps.

4.  Select **OK**.

# Step 5: Encrypt some columns in place

In this step, you'll encrypt the data stored in the **SSN** and **Salary** columns inside the server-side enclave, and then test a SELECT query on the data.

1.  Open a new SSMS instance and connect to your SQL Server instance **with** Always Encrypted enabled for the database connection.

- Start a new instance of SSMS.
- In the **Connect to Server** dialog, specify your server name, select an authentication method and specify your credentials.
- Select **Options \>\>** and select the **Always Encrypted** tab.
- Select the **Enable Always Encrypted (column encryption)** checkbox.
- Select **Enable secure enclaves**.
- Set **Protocol** to **None**.
- Select **Connect**.
- If you're prompted to enable Parameterization for Always Encrypted queries, select **Enable**.

2.  Using the same SSMS instance (with Always Encrypted enabled), open a new query window and encrypt the **SSN** and **Salary** columns by running the below queries.

In [ ]:
USE ADVENTUREWORKS
GO
ALTER TABLE [HumanResources].[Employees]
ALTER COLUMN [SSN] [char] (11) COLLATE Latin1_General_BIN2
ENCRYPTED WITH (COLUMN_ENCRYPTION_KEY = [CEK1], ENCRYPTION_TYPE = Randomized, ALGORITHM = 'AEAD_AES_256_CBC_HMAC_SHA_256') NOT NULL
WITH
(ONLINE = ON);

ALTER TABLE [HumanResources].[Employees]
ALTER COLUMN [Salary] [money]
ENCRYPTED WITH (COLUMN_ENCRYPTION_KEY = [CEK1], ENCRYPTION_TYPE = Randomized, ALGORITHM = 'AEAD_AES_256_CBC_HMAC_SHA_256') NOT NULL
WITH
(ONLINE = ON);

ALTER DATABASE SCOPED CONFIGURATION CLEAR PROCEDURE_CACHE;

Notice the ALTER DATABASE SCOPED CONFIGURATION CLEAR PROCEDURE\_CACHE statement to clear the query plan cache for the database in the above script. After you have altered the table, you need to clear the plans for all batches and stored procedures that access the table, to refresh parameters encryption information.

3.  To verify the SSN and Salary columns are now encrypted, open a new query window in the SSMS instance **without** Always Encrypted enabled for the database connection and execute the below statement. The query window should return encrypted values in the SSN and Salary columns. If you execute the same query using the SSMS instance with Always Encrypted enabled, you should see the data decrypted.

In [ ]:
SELECT [EmployeeID]
      ,[SSN]
      ,[FirstName]
      ,[LastName]
      ,[Salary]
  FROM [AdventureWorks].[HumanResources].[Employees]

# Step 6: Run rich queries against encrypted columns

Now, you can run rich queries against the encrypted columns. Some query processing will be performed inside your server-side enclave.

1.  In the SSMS instance **with** Always Encrypted enabled, make sure Parameterization for Always Encrypted is also enabled.

- Select **Tools** from the main menu of SSMS.
- Select **Options...**.
- Navigate to **Query Execution** \> **SQL Server** \> **Advanced**.
- Ensure that **Enable Parameterization for Always Encrypted** is checked.
- Select **OK**.

2.  Open a new query window, paste in and execute the below query. The query should return plaintext values and rows meeting the specified search criteria.

In [ ]:
DECLARE @SSNPattern [char](11) = '%6818';
DECLARE @MinSalary [money] = $1000;
SELECT * FROM [HR].[Employees]
WHERE SSN LIKE @SSNPattern AND [Salary] >= @MinSalary;

3\. Try the same query again in the SSMS instance that doesn't have Always Encrypted enabled, and note the failure that occurs.